# Planning

This notebook demonstrates planning using CoT.

In [1]:
#%pip install -qU langchain-google-vertexai

In [1]:
from dotenv import load_dotenv

load_dotenv("../genai_agents/keys.env");

In [2]:
from langchain_core.prompts import PromptTemplate
from langchain_google_vertexai import ChatVertexAI
from langchain_core.output_parsers import StrOutputParser

## 1. LLMs are improving ...

for many questions, they will do a chain-of-thought and come up with the right answer.

In [12]:
model = ChatVertexAI(model="gemini-1.5-flash")

def ask_question(question, chain_of_thought=False):
    if chain_of_thought:
        preamble = """
        Write down the steps to solve this question, and work through the steps
        one-by-one to come up with the answer
        """
    else:
        preamble = ""
   
    prompt_template = PromptTemplate.from_template(
        preamble + "{question}"
    )
    parser = StrOutputParser()
    chain = prompt_template | model | parser

    answer = chain.invoke({
        "question": question,
    })
    print(answer)

ask_question("Who is the Mayor of the capital of the third most populated state in the United States?")

Here's how to break down the question:

* **Third most populated state:** This would be Florida.
* **Capital:** The capital of Florida is Tallahassee.
* **Mayor:**  The current Mayor of Tallahassee is **John Dailey**. 

Let me know if you have other questions! 



## At this time, this doesn't work without explicit chain-of-thought